Acá detectamos la presencia de instancias de español mezclado con inglés no traducidas debidamente, así como de publicaciones nativamente en alemán. Por su bajo número, las eliminamos manualmente. 

In [5]:
from google.colab import files
uploaded = files.upload()


Saving articulos_upch_traducidos_con_editorial.csv to articulos_upch_traducidos_con_editorial (1).csv


In [2]:
!pip install pandas langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=aa18bec1505e5c8c602de1e44203cc8ae403ac41aad65d2bcafd48d61664247f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [9]:
import pandas as pd
from langdetect import detect_langs
from langdetect.lang_detect_exception import LangDetectException
import csv

# Reemplaza este nombre con el de tu archivo subido si es diferente
nombre_archivo = list(uploaded.keys())[0]
df = pd.read_csv(nombre_archivo, sep=';')

# Listas para clasificar las filas
solo_espanol = []
solo_ingles = []
mixto = []

def clasificar_idioma(texto):
    try:
        if pd.isnull(texto) or texto.strip() == "":
            return "desconocido", []
        idiomas = detect_langs(texto)
        lenguas = [(str(l.lang), l.prob) for l in idiomas if l.prob > 0.10]
        if "es" in [l[0] for l in lenguas] and "en" in [l[0] for l in lenguas]:
            return "mixto", lenguas
        elif "es" in [l[0] for l in lenguas]:
            return "espanol", lenguas
        elif "en" in [l[0] for l in lenguas]:
            return "ingles", lenguas
        else:
            return "otro", lenguas
    except LangDetectException:
        return "desconocido", []

# Clasificación fila por fila
for index, row in df.iterrows():
    resumen = row["abstract"]  # Cambia si tu columna se llama distinto
    categoria, idiomas_detectados = clasificar_idioma(resumen)
    if categoria == "espanol":
        solo_espanol.append(row)
    elif categoria == "ingles":
        solo_ingles.append(row)
    elif categoria == "mixto" or categoria == "otro":
        mixto.append(row)
        if categoria == "otro":
          print(f"Row {index}: Classified as 'otro', detected languages: {idiomas_detectados}")


# Guardar resultados
pd.DataFrame(mixto).to_csv("mixto.csv", index=False)

Row 483: Classified as 'otro', detected languages: [('fr', 0.9999949452861376)]
Row 654: Classified as 'otro', detected languages: [('de', 0.9999968064832473)]
Row 2089: Classified as 'otro', detected languages: [('de', 0.9999972319564372)]
Row 4502: Classified as 'otro', detected languages: [('de', 0.9999969442197374)]


Detectamos más idiomas que no se pudieron traducir con GoogleTranslator

In [ ]:
import pandas as pd
from langdetect import detect, DetectorFactory

# Para que los resultados sean reproducibles
DetectorFactory.seed = 0

# 2) Ruta a tu CSV
INPUT_CSV = r"C:\Users\DAVID 21\OneDrive\Documentos\9no ciclo\IMLE\TextMineML\results\tables\articulos_upch_traducidos_con_editorial.csv"
#OUTPUT_CSV = INPUT_CSV.replace(".csv", "_con_idioma.csv")

# 3) Carga el DataFrame
df = pd.read_csv(INPUT_CSV, sep=";", encoding="utf-8")

# 4) Función de detección de idioma
def detect_language(text: str) -> str:
    try:
        return detect(text)
    except:
        return "unknown"


# 6) Aplica la detección fila por fila
df["idioma_tit"] = df["title"].apply(detect_language)
df["idioma_abs"] = df["abstract"].apply(detect_language)



In [ ]:
# Imprime el total de idiomas detectados en cada columna
total_tit_langs = df["idioma_tit"].nunique()
total_abs_langs = df["idioma_abs"].nunique()
print(f"Total idiomas distintos en títulos: {total_tit_langs}")
print(f"Total idiomas distintos en abstracts: {total_abs_langs}\n")

# ---- Para títulos ----
print("=== TÍTULOS ===")
for lang in df["idioma_tit"].unique():
    idxs = df.index[df["idioma_tit"] == lang].tolist()
    count = len(idxs)
    if count == 0:
        continue
    if lang == "en":
        # Sólo imprime el conteo, sin índices ni primera fila
        print(f"Idioma: '{lang}' → {count} ocurrencia(s)")
        continue

    # Para cualquier otro idioma, imprime índices y primera ocurrencia
    print(f"\nIdioma: '{lang}' → {count} ocurrencia(s), índices: {idxs}")
    first = idxs[0]
    print(f" Primera fila [{first}]:")
    print("   titulo       :", df.at[first, "title"])
    print("   idioma_tit   :", df.at[first, "idioma_tit"])


# ---- Para abstracts ----
print("\n=== ABSTRACTS ===")
for lang in df["idioma_abs"].unique():
    idxs = df.index[df["idioma_abs"] == lang].tolist()
    count = len(idxs)
    if count == 0:
        continue
    if lang == "en":
        # Sólo imprime el conteo, sin índices ni primera fila
        print(f"Idioma: '{lang}' → {count} ocurrencia(s)")
        continue

    print(f"\nIdioma: '{lang}' → {count} ocurrencia(s), índices: {idxs}")
    first = idxs[0]
    print(f" Primera fila [{first}]:")
    print("   resumen      :", df.at[first, "abstract"])
    print("   idioma_abs   :", df.at[first, "idioma_abs"])


Total idiomas distintos en títulos: 11
Total idiomas distintos en abstracts: 6

=== TÍTULOS ===
Idioma: 'en' → 4912 ocurrencia(s)

Idioma: 'es' → 5 ocurrencia(s), índices: [155, 1586, 3454, 3586, 4654]
 Primera fila [155]:
   titulo       : Livedo reticularis y dengue
   idioma_tit   : es

Idioma: 'ro' → 17 ocurrencia(s), índices: [350, 672, 946, 1240, 1281, 1794, 2330, 2448, 2552, 2578, 3308, 3568, 3778, 3800, 3827, 4325, 4331]
 Primera fila [350]:
   titulo       : Hepatecellular carcinoma in Peru: a molecular description of an atypical clinical picture
   idioma_tit   : ro

Idioma: 'it' → 25 ocurrencia(s), índices: [632, 709, 1344, 1360, 1434, 1699, 1863, 1898, 2022, 2498, 2725, 2898, 3022, 3072, 3242, 3269, 3298, 3305, 3436, 3749, 3887, 4314, 4328, 4369, 4936]
 Primera fila [632]:
   titulo       : Multimorbidity
   idioma_tit   : it

Idioma: 'pt' → 7 ocurrencia(s), índices: [829, 1319, 1864, 2328, 3538, 3603, 4683]
 Primera fila [829]:
   titulo       : Fasciola hepatica cysteine 

Detectamos títulos y abstracts de longitud corta

In [5]:
import pandas as pd
INPUT_CSV = r"C:\Users\DAVID 21\OneDrive\Documentos\9no ciclo\IMLE\TextMineML\results\tables\articulos_upch_traducidos_con_editorial.csv"
#OUTPUT_CSV = INPUT_CSV.replace(".csv", "_con_idioma.csv")

df = pd.read_csv(INPUT_CSV, sep=";", encoding="utf-8")

#Contamos palabras en title y abstract
def contar_palabras(texto):
    if pd.isnull(texto) or texto.strip() == "":
        return 0
    return len(texto.strip().split())

# Aplicamos así la función a cada columna con apply
df['title_word_count'] = df['title'].apply(contar_palabras)
df['abstract_word_count'] = df['abstract'].apply(contar_palabras)

# Filtramos las filas donde title o abstract tienen menos de 4 palabras
filtro = (df['title_word_count'] < 4) | (df['abstract_word_count'] < 4)
filas_filtradas = df[filtro]

# Muestra los índices y los textos problemáticos
for idx, row in filas_filtradas.iterrows():
    print(f"Índice: {idx}")
    print(f"Title: {row['title']}")
    print(f"Abstract: {row['abstract']}")
    print("-" * 40)

Índice: 223
Title: Syphilis
Abstract: Syphilis is a sexually and vertically transmitted bacterial infection caused by the bacterium Treponema pallidum. Its prevalence is high in low-income and middle-income countries, and its incidence has increased in high-income countries in the last few decades among men who have sex with men. Syphilis is a major cause of adverse pregnancy outcomes in low-income and middle-income countries. Clinical features include a primary chancre at the point of inoculation, followed weeks later by the rash of secondary syphilis, a latent period, and in some cases, involvement of the eyes, CNS, and cardiovascular systems. It is diagnosed serologically. A single intramuscular dose of long-acting benzathine penicillin is recommended for people who have had syphilis for less than 1 year and longer courses for people with late latent syphilis. Control strategies include screening and treatment of all pregnant women, and targeted interventions for groups at high risk

In [7]:
import pandas as pd
INPUT_CSV = r"C:\Users\DAVID 21\OneDrive\Documentos\9no ciclo\IMLE\TextMineML\results\tables\articulos_upch_traducidos_con_editorial_editados_manual.csv"
#OUTPUT_CSV = INPUT_CSV.replace(".csv", "_con_idioma.csv")

df = pd.read_csv(INPUT_CSV, sep=";", encoding="utf-8")

#Contamos palabras en title y abstract
def contar_palabras(texto):
    if pd.isnull(texto) or texto.strip() == "":
        return 0
    return len(texto.strip().split())

# Aplicamos así la función a cada columna con apply
df['title_word_count'] = df['title'].apply(contar_palabras)
df['abstract_word_count'] = df['abstract'].apply(contar_palabras)

# Filtramos las filas donde title o abstract tienen menos de 4 palabras
filtro = (df['title_word_count'] < 4) | (df['abstract_word_count'] < 4)
filas_filtradas = df[filtro]

# Muestra los índices y los textos problemáticos
for idx, row in filas_filtradas.iterrows():
    print(f"Índice: {idx}")
    print(f"Title: {row['title']}")
    print(f"Abstract: {row['abstract']}")
    print("-" * 40)

Índice: 223
Title: Syphilis
Abstract: Syphilis is a sexually and vertically transmitted bacterial infection caused by the bacterium Treponema pallidum. Its prevalence is high in low-income and middle-income countries, and its incidence has increased in high-income countries in the last few decades among men who have sex with men. Syphilis is a major cause of adverse pregnancy outcomes in low-income and middle-income countries. Clinical features include a primary chancre at the point of inoculation, followed weeks later by the rash of secondary syphilis, a latent period, and in some cases, involvement of the eyes, CNS, and cardiovascular systems. It is diagnosed serologically. A single intramuscular dose of long-acting benzathine penicillin is recommended for people who have had syphilis for less than 1 year and longer courses for people with late latent syphilis. Control strategies include screening and treatment of all pregnant women, and targeted interventions for groups at high risk